# Scrapping Trip

To complete the data base of the hotels we are going to scrap tripadvisor website again. I took the code for scrapping from my previous project were I already had the code prepare and the list of unique hotels. 

In [1]:
import requests
import pandas as pd
from time import sleep
import numpy as np

from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options


import warnings
warnings.filterwarnings('ignore')

The following csv correspond to the list of hotels and the reviews from the previous scrapping

In [3]:
df = pd.read_csv("../Data/df_datos_trip.csv", sep=',', encoding='UTF-8', index_col=0)
pd.options.display.max_columns = None
df.head(5)

,Hotel,Trip_Rate,Trip_Location,Trip_Cleanliness,Trip_Service,Trip_Value
0,Rosewood Castiglion del Bosco,5.0,4.7,4.9,4.8,4.4
1,Grace Hotel,5.0,4.9,4.9,4.8,4.4
2,Waldorf Astoria Maldives Ithaafushi,5.0,4.9,4.9,4.8,4.5
3,Pickering House Inn,5.0,5.0,4.8,4.9,4.4
4,One&Only Reethi Rah,5.0,4.8,4.9,4.8,4.2


In [6]:
hotel_rev = df["Hotel"].unique().tolist()
print(hotel_rev)

['Rosewood Castiglion del Bosco', 'Grace Hotel', 'Waldorf Astoria Maldives Ithaafushi', 'Pickering House Inn', 'One&Only Reethi Rah', 'Royal Mansour Marrakech', 'Capella Ubud', 'The Lowell', 'Hôtel Madame Rêve', 'Rosewood Villa Magna', 'The Oberoi New Delhi', 'The Oberoi Udaivilas', 'Mandapa, a Ritz-Carlton Reserve', 'Wilderness Safaris Bisate Lodge', 'Portrait Firenze', 'Raffles Istanbul', 'The Oberoi Marrakech', 'Capella Hanoi', 'White Elephant Palm Beach', 'The Loutrel', 'Monasterio', 'Sani Asterias', 'Shangri-La the Shard', 'Coquillade Provence Resort & Spa', 'Taj Palace', 'Pendry Chicago', 'Nayara Tented Camp', 'Hotel Belmar', 'Royal Champagne Hôtel & Spa', 'Under Canvas Mount Rushmore', 'Riggs Washington D.C.', 'Waldorf Astoria Los Cabos Pedregal', 'The Oberoi Vanyavilas Wildlife Resort', 'Canaves Oia Epitome ', 'The Standard ', 'The Oberoi Amarvilas', 'Hotel Santa Caterina', "Raffles Grand Hotel d'Angkor", 'Nihi Sumba', 'La Réserve Paris', 'Hotel de la Ville', 'Il San Pietro di 

Below the code to do the scrapping on Tripadvisor Web

In [4]:
opciones= Options()
opciones.add_experimental_option('excludeSwitches', ['enable-automation'])
opciones.add_experimental_option('useAutomationExtension', False)
opciones.add_argument('--start-maximized') 
opciones.add_argument('user.data-dir=selenium') 
opciones.add_argument('--incognito')

In [9]:
hotels_link = {"Hotel": [],
                "Link_hotel": []} 

for hotel in hotel_rev: 
    driver = webdriver.Chrome(ChromeDriverManager().install())
    driver.implicitly_wait(8)


    driver.get("https://www.tripadvisor.com")
    driver.implicitly_wait(9)
              

    driver.find_element("css selector", "#onetrust-accept-btn-handler").click()
    sleep(7)

    driver.find_element("css selector", "#lithium-root > main > div.cBOoN > div.rGQXC._T.ivvQp > div > div > div:nth-child(1) > a > span.QLiHN.o.W > span").click()
    sleep(5)

    driver.find_element("css selector", "body > div.kaEuY > div > form > input.qjfqs._G.B-.z._J.Cj.R0").click()
    sleep(5)

    driver.find_element("css selector", "body > div.kaEuY > div > form > input.qjfqs._G.B-.z._J.Cj.R0").send_keys(f"{hotel}", Keys.TAB)
    sleep(6)

    driver.find_element("css selector", "#typeahead_results > a:nth-child(2) > div.EtzER > div.biGQs._P.fiohW.fOtGX").click()
    sleep(5)
        
    hotels_link["Hotel"].append(f"{hotel}")

    try:
        hotels_link["Link_hotel"].append(driver.find_element("css selector", "#ABOUT_TAB > div.ui_columns.MXlSZ > div:nth-child(2) > div.ssr-init-26f.pmndW > div.eeVey.S4.H3.f.u.QkOyE > a").get_attribute('href'))
    except: 
        hotels_link["Link_hotel"].append(np.nan)

    driver.quit()
        

As a result for the scrapping we obtain a list that we convert into a DataFrame

In [108]:
Hotel_link = pd.DataFrame(hotels_link, columns=["Hotel","Link_hotel"])
Hotel_link.head()

,Hotel,Link_hotel
0,Rosewood Castiglion del Bosco,https://www.tripadvisor.com/Commerce?p=TABAInd...
1,Grace Hotel,NaN
2,Waldorf Astoria Maldives Ithaafushi,https://www.tripadvisor.com/Commerce?p=TABAInd...
3,Pickering House Inn,NaN
4,One&Only Reethi Rah,https://www.tripadvisor.com/Commerce?p=TABAInd...


We can see that there are to many Nan values for the link column. 

In [103]:
nan_rows_trip = Hotel_link[Hotel_link['Link_hotel'].isna()]
nan_rows_trip.head()

,Hotel,Link_hotel
1,Grace Hotel,NaN
3,Pickering House Inn,NaN
7,The Lowell,NaN
8,Hôtel Madame Rêve,NaN
9,Rosewood Villa Magna,NaN


So we are going to do the scrapping in Google, for the hotels which didn't had a link in the Tripadvisor website, so first we create the list to then do the scrapping.

In [58]:
df_google_link = nan_rows_trip[["Hotel"]]
list_google_link = df_google_link["Hotel"].tolist()

In [61]:
hotels_link_google = {"Hotel": [],
                "Link_hotel": []} 

for hotel in list_google_link: 
    driver = webdriver.Chrome(ChromeDriverManager().install())
    driver.implicitly_wait(8)
            
    driver.get("https://www.google.com")
    driver.implicitly_wait(7)

    driver.find_element("css selector", "#L2AGLb > div").click()
    sleep(4)

    driver.find_element("css selector", "body > div.L3eUgb > div.o3j99.ikrT4e.om7nvf > form > div:nth-child(1) > div.A8SBwf > div.RNNXgb > div > div.a4bIc > input").send_keys(f"{hotel}", Keys.ENTER)
    sleep(6)
        
    hotels_link_google["Hotel"].append(hotel)
        
    try:
        hotels_link_google["Link_hotel"].append(driver.find_element('xpath', '/html/body/div[7]/div/div[11]/div[2]/div/div/div[2]/div/div[1]/div/div[2]/div[1]/div/div/div/div[1]/a').get_attribute('href'))
    except: 
        hotels_link_google["Link_hotel"].append(np.nan)

        driver.quit()
    

In [55]:
driver = webdriver.Chrome(ChromeDriverManager().install())
driver.implicitly_wait(8)
            
driver.get("https://www.google.com")
driver.implicitly_wait(7)

driver.find_element("css selector", "#L2AGLb > div").click()
sleep(4)

driver.find_element("css selector", "body > div.L3eUgb > div.o3j99.ikrT4e.om7nvf > form > div:nth-child(1) > div.A8SBwf > div.RNNXgb > div > div.a4bIc > input").send_keys("Cap Maison", Keys.ENTER)
sleep(8)
        

Link_hotel = driver.find_element('xpath', '/html/body/div[7]/div/div[11]/div[2]/div/div/div[2]/div/div[1]/div/div[2]/div[1]/div/div/div/div[1]/a').get_attribute('href')




In [102]:
hotels_link_2 = pd.DataFrame(hotels_link_google, columns=["Hotel","Link_hotel"])
hotels_link_2

,Hotel,Link_hotel
0,Grace Hotel,https://aubergeresorts.com/gracehotel/?utm_sou...
1,Pickering House Inn,http://www.pickeringhousewolfeboro.com/
2,The Lowell,https://www.lowellhotel.com/?utm_source=mybusi...
3,Hôtel Madame Rêve,https://madamereve.com/?utm_source=google&utm_...
4,Rosewood Villa Magna,https://www.rosewoodhotels.com/en/villa-magna
...,...,...
76,Lion Sands Game Reserve,http://www.lionsands.com/
77,The Stafford,http://www.thestaffordlondon.com/
78,Cap Maison,https://capmaison.com/
79,Tensing Pen,https://www.tensingpen.com/


In [111]:
Hotel_link.fillna(hotels_link_2, inplace=True)

In [114]:
links_hotels = Hotel_link.copy()
links_hotels.head()

,Hotel,Link_hotel
0,Rosewood Castiglion del Bosco,https://www.tripadvisor.com/Commerce?p=TABAInd...
1,Grace Hotel,http://www.pickeringhousewolfeboro.com/
2,Waldorf Astoria Maldives Ithaafushi,https://www.tripadvisor.com/Commerce?p=TABAInd...
3,Pickering House Inn,https://madamereve.com/?utm_source=google&utm_...
4,One&Only Reethi Rah,https://www.tripadvisor.com/Commerce?p=TABAInd...


I save the data in the folder Data

In [116]:
links_hotels.to_csv("../Data/df_hotel_link.csv")